---
title: "Maggy Distributed Training on MNIST Dataset using Python Kernel"
date: 2022-02-16
type: technical_note
draft: false
---

Maggy enables you to train with Tensorflow distributed optimizers.
Using Maggy, you have to make minimal changes in train your model in a distributed fashion. 

Make sure you have the latest Maggy installation and that you are running this notebook with a Python kernel. 

In [25]:
from maggy.experiment import experiment
from maggy.experiment_config.tf_distributed import TfDistributedConfig

import tensorflow as tf
from tensorflow import keras
import numpy as np

### 1. Model definition

Let's define the model we want to train. The layers of the model have to be defined in the \_\_init__ function.

Do not instantiate the class, otherwise you won't be able to use Maggy.

In [26]:
class NeuralNetwork(tf.keras.Model):

    def __init__(self, nlayers):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(28, 2, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.d1 = keras.layers.Dense(32, activation='relu')
        self.d2 = keras.layers.Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = NeuralNetwork

### 2. Dataset creation

You can create the dataset here and pass it to the TfDistributedConfig, or creating it in the training function.

In this example, we are downloading the dataset from tensorflow.

In [27]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.reshape(x_train, (60000, 28, 28, 1))
x_test = np.reshape(x_test, (10000, 28, 28, 1))

### 3. Defining the training function

The programming model is that you wrap the code containing the model training inside a wrapper function. Inside that wrapper function provide all imports and parts that make up your experiment.

The function should return the metric that you want to optimize for. This should coincide with the metric being reported in the Keras callback (see next point).
You can return the metric list, in this case only the loss element will be printed.

In [31]:
def training_function(model, train_set, test_set, hparams):
    
    # Define training parameters
    num_epochs = 10
    batch_size = 256
    learning_rate = 0.1

    criterion = keras.losses.SparseCategoricalCrossentropy()
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate,momentum=0.9,decay=1e-5)
    
    model = model(nlayers = hparams['nlayers'])
    
    model.compile(optimizer=optimizer, loss=criterion, metrics=["accuracy"])
    
    model.fit(train_set[0],
              train_set[1],
              batch_size=batch_size,
              epochs=num_epochs,
              )

    print("Testing")
    
    loss = model.evaluate(
        test_set[0],
        test_set[1],
        batch_size=32)
    
    return loss

### 4. Configuring the experiment

In order to use maggy distributed training, we have to configure the training model, we can pass it to TfDistributedConfig.
the model class has to be an implementation of __tf.keras.Model__.
We can also define __train_set__, __test_set__ and eventually the __model_parameters__. __model_parameters__ is a dictionary
containing the parameters to be used in the \_\_init__ function of your model.

In [32]:
#define the constructor parameters of your model
model_parameters = {
    'nlayers': 2
}

#pass the model parameters in the last 
config = TfDistributedConfig(name="tf_test", 
                             model=model, 
                             train_set=(x_train, y_train), 
                             test_set=(x_test, y_test),
                             hparams=model_parameters
                            )

#### 5. Run distributed training

Finally, we are ready to launch the maggy experiment. You just need to pass 2 parameters: the training function and the configuration variable we defined in the previous steps.

In [33]:
result = experiment.lagom(training_function, config)

2022-02-16 10:52:47,768 INFO: Using MirroredStrategy with devices ('/device:CPU:0',)
2022-02-16 10:52:47,769 INFO: Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO
2022-02-16T10:52:47.771477 (0/0): train_set or test_set is null, data has not been sharded. 

2022-02-16T10:52:47.771691 (0/0): index of slice 0 

2022-02-16T10:52:47.771738 (0/0): Starting distributed training. 

Epoch 1/10
235/235 [==============================] - 13s 43ms/step - loss: 0.7543 - accuracy: 0.7819
Epoch 2/10
235/235 [==============================] - 10s 41ms/step - loss: 0.1528 - accuracy: 0.9553
Epoch 3/10
235/235 [==============================] - 10s 41ms/step - loss: 0.0815 - accuracy: 0.9762
Epoch 4/10
235/235 [==============================] - 10s 43ms/step - loss: 0.0488 - accuracy: 0.9855
Epoch 5/10
235/235 [==============================] - 10s 43ms/step - loss: 0.0378 - accuracy: 0.9878
Epoch 6/10
235/235 [=========

In [34]:
print('loss: {}'.format(result))

loss: 0.09255693107843399
